In [2]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader
from collections import Counter

In [3]:
# Préparation des données
#text = "hello world hello neural networks hello pytorch"
# Remplacez 'chemin/vers/votre/fichier.txt' par le chemin réel vers votre fichier
with open('/content/lafontaine_short.txt', 'r', encoding='utf-8') as f:
    text = f.read()
#print(text)
chars = tuple(set(text))
print(chars)
int2char = dict(enumerate(chars))
print(int2char)
char2int = {char: idx for idx, char in int2char.items()}
print(char2int)

('5', 't', 'Y', ':', 'Q', 'd', 'è', 'S', 'v', 'H', 'y', ',', 'k', 'Ç', '2', "'", 'j', 'A', 'O', 'i', '.', 'g', 'h', 'L', 'u', '-', 'R', 'p', 'ó', 'B', 'ù', 'G', 'n', 'N', 'c', 'U', 'Z', 'À', 's', '1', 'ç', 'é', 'T', 'œ', 'e', 'I', '0', 'w', 'z', 'E', 'f', 'o', 'C', 'M', '4', '!', '"', 'P', 'à', 'q', 'â', 'a', ' ', 'F', '?', 'ê', 'm', 'r', '\n', 'û', 'b', '9', 'D', 'É', 'V', 'l', 'J', 'x', '3', 'ô')
{0: '5', 1: 't', 2: 'Y', 3: ':', 4: 'Q', 5: 'd', 6: 'è', 7: 'S', 8: 'v', 9: 'H', 10: 'y', 11: ',', 12: 'k', 13: 'Ç', 14: '2', 15: "'", 16: 'j', 17: 'A', 18: 'O', 19: 'i', 20: '.', 21: 'g', 22: 'h', 23: 'L', 24: 'u', 25: '-', 26: 'R', 27: 'p', 28: 'ó', 29: 'B', 30: 'ù', 31: 'G', 32: 'n', 33: 'N', 34: 'c', 35: 'U', 36: 'Z', 37: 'À', 38: 's', 39: '1', 40: 'ç', 41: 'é', 42: 'T', 43: 'œ', 44: 'e', 45: 'I', 46: '0', 47: 'w', 48: 'z', 49: 'E', 50: 'f', 51: 'o', 52: 'C', 53: 'M', 54: '4', 55: '!', 56: '"', 57: 'P', 58: 'à', 59: 'q', 60: 'â', 61: 'a', 62: ' ', 63: 'F', 64: '?', 65: 'ê', 66: 'm', 67: 

In [4]:
# Encodage du texte
encoded_text = np.array([char2int[char] for char in text])
print(encoded_text)

[57 44 67 ... 38 62 55]


In [5]:
# Création d'un dataset
class CharDataset(Dataset):
    def __init__(self, data, sequence_length=10):
        self.data = data
        self.sequence_length = sequence_length

    def __len__(self):
        return len(self.data) - self.sequence_length

    def __getitem__(self, index):
        return (
            torch.tensor(self.data[index:index+self.sequence_length]),
            torch.tensor(self.data[index+1:index+self.sequence_length+1])
        )

dataset = CharDataset(encoded_text, sequence_length=10)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)
print(dataset)

In [6]:
 #Définition du modèle
class CharRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(CharRNN, self).__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.rnn = nn.RNN(input_size, hidden_size, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        out, hidden = self.rnn(x, hidden)
        out = self.fc(out)
        return out, hidden

    def init_hidden(self, batch_size):
        return torch.zeros(self.n_layers, batch_size, self.hidden_size)



In [7]:
# Instanciation du modèle
model = CharRNN(len(chars), 128, len(chars))
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)


In [8]:
# Entraînement du modèle
def train(dataloader, model, loss_fn, optimizer, epochs=10):
    for epoch in range(epochs):
        hidden = model.init_hidden(batch_size=2)
        for batch, (X, y) in enumerate(dataloader):
            batch_size = X.size(0)  # Obtient la taille du lot actuel
            hidden = model.init_hidden(batch_size=batch_size)  # Initialise l'état caché en fonction de la taille du lot actuel
            # Préparation de l'entrée et des cibles
            X = nn.functional.one_hot(X, num_classes=len(chars)).float()
            y = y.view(-1)
            hidden = hidden.data

            # Calcul de la prédiction et de la perte
            pred, hidden = model(X, hidden)
            loss = loss_fn(pred.view(-1, len(chars)), y)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')


In [10]:
# Lancer l'entraînement
train(dataloader, model, loss_fn, optimizer, epochs=100)

Epoch 0, Loss: 2.7587392330169678
Epoch 10, Loss: 2.3589396476745605
Epoch 20, Loss: 2.2136359214782715
Epoch 30, Loss: 3.343876361846924
Epoch 40, Loss: 2.025679349899292
Epoch 50, Loss: 1.3783395290374756
Epoch 60, Loss: 2.1365103721618652
Epoch 70, Loss: 2.821737766265869
Epoch 80, Loss: 2.464508533477783
Epoch 90, Loss: 2.215266704559326


In [14]:
# Après l'entraînement
# 'model' est votre instance de modèle entraînée
# 'chemin_sauvegarde' est le chemin du fichier où sauvegarder l'état du modèle

chemin_sauvegarde = '/content/mon_modele_conv.pkl'
torch.save(model.state_dict(), chemin_sauvegarde)


In [9]:
device='cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [15]:
def generate(model, start_str='h', length=100, temperature=1.0):
    model.eval()
    chars = [c for c in start_str]
    hidden = model.init_hidden(1)

    for _ in range(length):
        input_seq = torch.tensor([[char2int[c] for c in chars[-model.n_layers:]]], dtype=torch.long)

        # S'assurer que num_classes correspond à la taille du vocabulaire
        num_classes = len(char2int)  # Correction ici pour utiliser la taille correcte
        input_seq = nn.functional.one_hot(input_seq, num_classes=num_classes).float()

        with torch.no_grad():
            output, hidden = model(input_seq, hidden)

        output_dist = output.data.view(-1).div(temperature).exp()
        top_char = torch.multinomial(output_dist, 1)[0]
        predicted_char = int2char[top_char.item()]
        chars.append(predicted_char)

    return ''.join(chars)


In [20]:
# Générer du texte avec le modèle
generated_text = generate(model, start_str='qu\'est-e qu\'on commade ?', length=400)
print(generated_text)

qu'est-e qu'on commade ?
Perkoek, lunchetuest, ireti,rel'e t'On ?ientetmiest pork, ça Viren er chabs tei, Insur Semen fommériyumevilv erk, un art pois, Siy pessierrles ,, almmakur ?
Personne 1:e ent muyujuni,rier". Me es enta fitr'eci, Silvaire, pouisfiâs Sourk, Oire veman. Ouvey, mune l'ainte hoututak,éle je es fplrst, Sbount.
Personne 2: Sente ast filr, unt entebt Saunivouk, bonne 1: On, daite,.ten. "xéihobk enoune, un


In [24]:
#Charger le modèle
#model = CharRNN(input_size, hidden_size, output_size, n_layers)  # Assurez-vous que l'architecture correspond
model = CharRNN(len(chars), 128, len(chars))
model.load_state_dict(torch.load('/content/mon_modele_conv.pkl'))
model.eval()  # Passer le modèle en mode évaluation


CharRNN(
  (rnn): RNN(80, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=80, bias=True)
)

In [23]:
# Générer du texte avec le modèle
generated_text = generate(model, start_str='gondole', length=400)
print(generated_text)

gondole 2: Ouitnh On tuxri àte ?
Personne 1: Sunelmbini, l'hons ?
Personne 1: Il'as, li l'he seert ? Sles coun pomeix bramplurche ?
Personne 1: Moptes. Ourk, je doukest ?ient, ?
Personne 2: Sbienomm fup?
Personne 2: T'heintirturstriticortbe ma létiyend moitieiren,, pnenté, je Veisl'ed, pnémait, j'adommalr de axe,, je, ?
Personne 2: J'hafón re toukekmantéunne xe d'hé, vament cixibrt, nhevarchéins.
Personn
